描述：

    根据输入数据和对应的label进行训练；
    观察收敛之后的网络的权重是否正确的权重之间的差距。
    [6.2.4 学习卷积核](https://zh-v2.d2l.ai/chapter_convolutional-neural-networks/conv-layer.html) 

In [30]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

真实卷积核

In [3]:
K = torch.tensor([[1.0, -1.0]])

In [4]:
def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [5]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

## 学习卷积核

In [6]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 5.400
epoch 4, loss 1.118
epoch 6, loss 0.274
epoch 8, loss 0.082
epoch 10, loss 0.028


In [7]:
conv2d.weight.data.shape

torch.Size([1, 1, 1, 2])

优化器

In [8]:
net = nn.Sequential(conv2d)

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [10]:
loss = nn.MSELoss()

In [11]:
from torch.utils import data

In [12]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器
    Defined in :numref:`sec_linear_concise`"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [30]:
X.shape

torch.Size([1, 1, 6, 8])

In [14]:
data_arrays = [X.view(6,1,1,8),Y.view(6,1,1,7)]

In [15]:
data_loader = load_array(data_arrays,2,False)

In [16]:
for x,y in data_loader:
    print(x,y)
    print('*'*20)

tensor([[[[1., 1., 0., 0., 0., 0., 1., 1.]]],


        [[[1., 1., 0., 0., 0., 0., 1., 1.]]]]) tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.]]],


        [[[ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]])
********************
tensor([[[[1., 1., 0., 0., 0., 0., 1., 1.]]],


        [[[1., 1., 0., 0., 0., 0., 1., 1.]]]]) tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.]]],


        [[[ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]])
********************
tensor([[[[1., 1., 0., 0., 0., 0., 1., 1.]]],


        [[[1., 1., 0., 0., 0., 0., 1., 1.]]]]) tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.]]],


        [[[ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]])
********************


In [27]:
num_epochs = 16
for epoch in range(num_epochs):
    for _data, _label in data_loader:
        l = loss(net(_data) ,_label)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(_data), _label)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000004
epoch 2, loss 0.000004
epoch 3, loss 0.000004
epoch 4, loss 0.000004
epoch 5, loss 0.000003
epoch 6, loss 0.000003
epoch 7, loss 0.000003
epoch 8, loss 0.000003
epoch 9, loss 0.000003
epoch 10, loss 0.000003
epoch 11, loss 0.000003
epoch 12, loss 0.000002
epoch 13, loss 0.000002
epoch 14, loss 0.000002
epoch 15, loss 0.000002
epoch 16, loss 0.000002


In [28]:
net[0].weight.data

tensor([[[[ 0.9974, -0.9974]]]])

In [26]:
net[0].weight.data

tensor([[[[ 0.9960, -0.9961]]]])